# Support Vector Machines

## Linear SVM Classification
<img src="./images/comparison_margin.png" width="1000">

Normal classifiers on left hand side, some are fine for the dataset, but on new data it will be bad (dashed line doesn't even seperate classes, and other lines are too close to the data). On right had side is the Support Vector Machine Classifier. You can see that this classifier allows the most space in-between the data, this is called large margin classification.

Note: SVMs are sensitive to the ferature scales, as you can see in this figure below, in the left plot, the vertical scale is much larger than the horizontal scale, so the widest possible street is close to horizontal. After feature scaling (e.g. using Scikit-Learns's `StandardScaler`) the decision boundary is the right plot looks much better

<img src="./images/comparison_feature_scales.png" width="1000">

___

## Soft Margin Classification
If we want the margin to be on the right hand size rather than the middle we will use hard margin classification. Issues with this approach

- only works if data is linearly separable
- sensitive to outliers

<img src="./images/outlier_softmargin.png" width="1000">

The left of this figure shows what the iris dataset would look like with an additional outlier. It is impossible to find a hard margin. On the right, the decision boundary ends up very different from the figure we have seen previously without the outlier. This means that the classifier won't generalize well.

To avoid these problems, we can use a more flexible model. We need to find a good balance between keeping the gap as large as possible and limiting the margin violations (i.e., instances that end up in the middle or on the wrong side of the "road"). This is called soft margin classification

`C` is a hyperparameter when creating a SVM model using Scikit-Learn. If we set it to a low value, then we end up with the model on the left. With a high model, we get the model ont he right. Margin violations are bad. It's usually better to have few of them. however, in this case the model on the left has a lot of margin violations but will probably generalize better.

<img src="./images/hyperparam.png" width="1000">

Tip: If the SVM model is overfitting, you can try to reduce C

___

The following code will load the dataset, scale the features and then trains a linear SVM model to detect Iris virginica flowers.

In [18]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris virginica

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])

svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [19]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

(Unlike Logistic  Regression classifiers, SVM classifiers do not output probabilities for each class).

# Nonlinear SVM Classification

linear SVM classifiers are efficient and work well for many cases, but many datasets are not even close to being linearly seperable. Adding more features can help handle non-linear datasets. e.g. left plot has one feature, which as you can see makes it not linearly seperable. But when you add a feature, it is.

<img src="./images/features.png" width="1000">

To implement this idea using Scikit-Learn, create a Pipeline containing a `PolynomialFeatures` transformer, followed by a `StandardScaler` and a `LinearSVC`. Let's test this out with the moons dataset: this is a toy dataset for binary classification in which the data points are shaped as two interleaving half circles.

<img src="./images/moon.png" width="500">

In [32]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
])

polynomial_svm_clf.fit(X, y)

/home/alpharaoh/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

## Polynomial Kernel

Adding polynomial features is simple to implement and can work great with all sorts of ML algos. However, at a low poly degree, this method cannot deal with very complex datasets, and with high degree it creates a huge number of features, making model too slow.

using SVM's you can apply a kernel trick. The kernel trick algomakes it possible to get the same result as if you had added many polynomial features, without actually having to add them. This trick is implemented by the SVC class.

In [39]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])

poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

In [41]:
poly_kernel_svm_clf.predict([[0.5, 0.]])

array([1])

This code trains the SVM classifier using a third-degree poly kernel. IT is represented on the left of figure below. If model is overfitting, reduce the poly degree, if underfitting , increase it. The hyperparameter coef0 controls how much the model is influenced by high-degree polynomials vs low-degree polynomials.

<img src="./images/kernel.png" width="1000">

Tip: A common approach to finding the right hyperparam values to use grid search. It is often faster to first do a very coarse grid search, then a finer grid search around the best values found.

# Similarity Features

Another way to tackle nonlinear problems is to add features computed using a similarity function, which measures how much each instance resembles a particular __landmark__, e.g. the 1D dataset discussed earlier and add two landmarks to it at $x_1=-2$ and $x_2=1$ (left of plot below). Next, let's define the similarity function to be the Guassian __Radial Basis Function (RBF)__ with $\gamma = 0.3$

$\phi_\gamma(x, l) = exp(-\gamma||x-l||^2)$

This is a bell-shaped function varying from 0 (very far from landmark) to 1 (at the landmark). Now we can compute the new features. e.g. look at $x_1=-1$: it is located at a distance of 1 from the landmark and 2 from the second landmark. Therefore its new features are $x_2 = exp(-0.3 * 1^2) \approx 0.74$ The plot on the right shows the transformed dataset (dropping the original features). Now, you can it is linearly seperable.

<img src="./images/poly_kernel_g.png" width="1000">

The simplest approach to select landmarks are to create a landmark at the location of each and every instance in the dataset. Doing that creates manu dimensions and this increase the chances that the transformed training set will be linearly seperable. The downside is that a training set with m instance and n features gets transformed into a training set with m instances and m features. If your training set is very large, you end up with an equally large number of features.

## Guassian RBF Kernel

Just like polynomial features method, the similarity features method can be useful with any Machine Learning algo, but may be computationally expensive to compute all the additional features, especially on large training sets. Once again the kernel trick does its SVM magic, making it possible to obtain a similiar result as if you had added many similarity features. Let's try the SVC class with the Guassian RBF kernel.

In [43]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])

rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

This model is represented at the bottom left of the figure below. The other plots show models trained with different values of hyperparam gamma ($\gamma$) and C.
<br><br>
Increasing